### Load model created by analyse_docs(...)

In [10]:
import os.path
import asyncio
import pickle as pkl

from bertopic import BERTopic

topic_model = BERTopic.load("/home/s371513/ernie/model")

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: '../model'.

### Label Topics

In [ ]:
custom_topics = {}

for i in range(len(topic_model.get_topic_info())):
    topic = input("----------------------------\nTopics already created: \n{}\n----------Topic {}----------\n{}\n----------------------------".format(set(custom_topics.values()), str(i),topic_model.get_topic(i)))
    custom_topics[i] = topic

with open('custom_topics.pkl', 'wb+') as f:
    pkl.dump(custom_topics, f)

### Match Topics

In [ ]:
    if os.path.isfile(os.path.join(cfg.base_path(), "custom_topics.pkl")):
        with open(os.path.join(cfg.base_path(), "custom_topics.pkl"), 'rb') as f:
            print("Loading custom topics...")
            topic_dictionary = pkl.load(f)

            print("Reading texts...")
            file_names, _ = read_text("preprocessed.txt")

            print("Applying topics to files in batches of {}...".format(chunk_size))
            file_chunks = [file_names[i:i + chunk_size] for i in range(0, len(file_names), chunk_size)]

            c = 1
            for file_chunk in file_chunks:
                print("-" * 16)
                print("Batch {} of {}".format(c, len(file_chunks)))
                print("-" * 16)
                loop = tqdm(file_chunk)

                corrupted_files = 0

                for topic_index, doc_file in enumerate(loop):
                    with open(os.path.join(cfg.gdelt_out(), doc_file), "rb") as d:
                        try:
                            document = pkl.load(d)
                            d.close()
                            document.topic_information = topic_dictionary[topic_index]
                            document.save_document()

                        except EOFError as _:
                            corrupted_files += 1
                            loop.set_postfix_str("Coruppted files: {}".format(corrupted_files))

                        d.close()
                c += 1
    else:
        print("No file of custom topics found, execute label_topics() first!")